## Text Classifier Example
Description: Given two poems, classify the poems as either being written by Edgar Allen Poe or Rober Frost
#### Resources 
* [Processed EAP Poems](https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt)
* [Processed RF Poems](https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt)

In [1]:
import sys
sys.path.append('D:\\GitHub\\reasearch-work\\natural-language-processing\\lib')

In [2]:
import copy
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score
import utils

#### Process the poems
The poems are plain text documents, so before we can build a classifier, we need to transform the documents into rows in
a dataset. In order to do this, we must accomplish the following:

* Read the plain text documents in
* Transform each line of the poems into a row
* Label the rows
* Combine the rows into a single dataset
* Split the dataset into a training set and a testing set

For the purposes of the above, we select `EAP` to denote rows from an Edgar Allen Poe poem, and `RF` to denote a Robert
Frost poem.

In [14]:
eap_poem = utils.get_poem('https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt').split('\n')
rf_poem = utils.get_poem('https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt').split('\n')

eap_poem = utils.label_data(eap_poem, 0)
rf_poem = utils.label_data(rf_poem, 1)

samples = np.array(utils.flatten([eap_poem, rf_poem]))
train_X, test_X, train_y, test_y = train_test_split(samples[:, 0], samples[:, -1], random_state=31337)
train_y = [int(y) for y in train_y]
test_y = [int(y) for y in test_y]

#### Build up mapping of words
In order to classify words, we need to index every word in the training set. We do this by tokenizing each row, and then mapping unique words to a unique index. Keep in mind we are only mapping the *training set* which means there's a statistical likelihood we'll encounter words that don't fit into our mapping when we run our classifier against the testing set. At a high-level, this means doing the following for each observation in the training set:

* Clean up punctuation, and convert to lower case
* Split each observation into individual words
* For each word, map the word to a unique index 1..n
* Map the key "NOT_SEEN" to the index 0
* Apply the map to each row in train and test

In [17]:
def token_helper(words):
    tokens = []
    for word in words:
        row = utils.tokenize(word.rstrip())
        tokens.append(row)
    return tokens

In [18]:
train_tokens = token_helper(train_X)
test_tokens = token_helper(test_X)
word_map = utils.index_words(train_tokens)
train_values = utils.word_to_idx(word_map, copy.deepcopy(train_tokens))
test_values  = utils.word_to_idx(word_map, copy.deepcopy(test_tokens))

#### Train the Markov Model
Now that we have the dataset mapped, we need to train the Markov Model. Training a Markov model involves computing the probability of a sequence given an initial state $\pi$ and a transition matrix $A$. Here is the calculation we need to make:

$log p(s_{1..T}) = log\pi_{s_1} \prod\limits_{t=2}^T logA_{s_ts_{t - 1}}$

We can estimate $\pi$ as:

$\hat{\pi} = \frac{count(s_1 = i) + 1}{count(num\_sequences) + N}$

We can estimate $A$ as:

$\hat{A_{ij}} = \frac{count(i \rightarrow j) + 1}{count(i) + M}$

Where $i$ is the previous state and $j$ is the state we are transitioning to. Put another way, we are evaluating the number of times we see a given sequence against the number of times we see the word by itself. Note also that we are smoothing the estimates so that we don't encounter 0 probabilities in the presence of previously unseen sequences.


In [97]:
class MarkovModel():
    def __init__(self, tokens_idx, V):
        self.tokens = tokens_idx
        self.V = V
        self.A0 = np.ones((self.V, self.V))
        self.pi0 = np.ones(self.V)
        self.A1 = np.ones((self.V, self.V))
        self.pi1 = np.ones(self.V)
        self.A = None
        self.pi = None
        self.priors = [0., 0.]
        self.n_classes = len(self.priors)
    
    def compute_counts(self, labels):        
        for row, label in zip(self.tokens, labels):
            prev_idx = None
            for token in row:
                curr_idx = token
               
                if not prev_idx:
                    if label == 0:
                        self.pi0[curr_idx] += 1
                    else:
                        self.pi1[curr_idx] += 1
                else:
                    if label == 0:
                        self.A0[prev_idx, curr_idx] += 1
                    else:
                        self.A1[prev_idx, curr_idx] += 1
                prev_idx = curr_idx
        
    def compute_priors(self, labels):
        count_0 = sum(label == 0 for label in labels)
        count_1 = sum(label == 1 for label in labels)
        total_labels = len(labels)
        
        prior_0 = np.log(count_0 / total_labels)
        prior_1 = np.log(count_1 / total_labels)
        self.priors = [prior_0, prior_1]
    
    def compute_log_likelihood(self, observation, class_label):
        A = self.A[class_label]
        pi = self.pi[class_label]
        
        prev_idx = None
        prob = 0.
        
        for idx in observation:
            if not prev_idx:
                prob += pi[idx]
                continue
            prob += A[prev_idx, idx]
            prev_idx = idx
        return prob
    
    def normalize_pi(self, pi):
        return np.log(pi / pi.sum())
    
    def normalize_A(self, A):
        return np.log(A / A.sum(axis=1, keepdims=True))
    
    def predict(self, observations):
        self.A = [self.A0, self.A1]
        self.pi = [self.pi0, self.pi1]
        predictions = np.zeros(len(observations))
        
        for i, observation in enumerate(observations):
            posteriors = [self.compute_log_likelihood(observation, class_label) \
                          + self.priors[class_label] for class_label in range(self.n_classes)]
            prediction = np.argmax(posteriors)
            predictions[i] = prediction
        return predictions
    

In [98]:
model = MarkovModel(train_values, len(word_map))

In [99]:
model.compute_counts(train_y)
model.pi0 = model.normalize_pi(model.pi0)
model.pi1 = model.normalize_pi(model.pi1)
model.A0 = model.normalize_A(model.A0)
model.A1 = model.normalize_A(model.A1)
model.compute_priors(train_y)

In [100]:
model.A

In [101]:
p_train = model.predict(train_values)
print('TRAIN_ACCURACY: ', np.mean(p_train == train_y))

TRAIN_ACCURACY:  0.74984520123839


In [102]:
p_test = model.predict(test_values)
print('TEST_ACCURACY: ', np.mean(p_test == test_y))

TEST_ACCURACY:  0.7087198515769945


In [103]:
cm = confusion_matrix(train_y, p_train)
cm

array([[ 174,  352],
       [  52, 1037]], dtype=int64)

In [104]:
cm = confusion_matrix(test_y, p_test)
cm

array([[ 53, 139],
       [ 18, 329]], dtype=int64)

In [105]:
f1_score(train_y, p_train)

0.8369652945924132

In [106]:
f1_score(test_y, p_test)

0.807361963190184